In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('eng_-french.csv')

In [39]:
df.head()

,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


In [120]:
batch_size = 64
epochs = 100
latent_dim = 256

In [108]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [109]:
index = 0
for i in df["English words/sentences"]:
    index+=1
    input_texts.append(i)
    for j in i:
        if j not in input_characters:
            input_characters.add(j)
    if index > len(df)/8:
        print(index)
        break

21953


In [110]:
index = 0
for i in df["French words/sentences"]:
    index+=1
    target_texts.append(i)
    for j in i:
        if j not in target_characters:
            target_characters.add(j)
    if index > len(df)/8:
        print(index)
        break

21953


In [111]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

In [112]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

In [113]:
max_encoder_seq_length = max([len(i) for i in input_texts])
max_decoder_seq_length = max([len(i) for i in target_texts])

In [117]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 21953
Number of unique input tokens: 74
Number of unique output tokens: 100
Max sequence length for inputs: 18
Max sequence length for outputs: 57


In [118]:
input_token_index = dict([(i, j) for j, i in enumerate(input_characters)])
target_token_index = dict([(i, j) for j, i in enumerate(target_characters)])

In [119]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

In [77]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [79]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [83]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=1,
    validation_split=0.2,
)
model.save("m1")

275/275 [==============================] - 105s 367ms/step - loss: 0.1002 - accuracy: 0.9682 - val_loss: 0.6140 - val_accuracy: 0.8641


INFO:tensorflow:Assets written to: m1\assets


INFO:tensorflow:Assets written to: m1\assets


# тут было где-то 50 эпох

In [84]:
model = keras.models.load_model("m1")

In [85]:
encoder_inputs = model.input[0] 
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [86]:
reverse_input_char_index = dict((i, j) for j, i in input_token_index.items())
reverse_target_char_index = dict((i, j) for j, i in target_token_index.items())

In [92]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #target_seq[0, 0, target_token_index["\t"]] = 1.0
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]
    return decoded_sentence

In [103]:
import random

In [116]:
for i in range(5):
    seq_index = random.randint(0,21953)
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("Стало:",input_texts[seq_index],"==>",decoded_sentence)
    print("Было:",df["English words/sentences"][seq_index],"=",df["French words/sentences"][seq_index],"\n")

Стало: Run for it! ==> auvez-vous !                                              
Было: Run for it! = Prenez vos jambes à votre cou ! 

Стало: That part is good. ==> a tière est courte.                                       
Было: That part is good. = Cette partie est bonne. 

Стало: The roof leaks. ==> e chien est mauvaise.                                     
Было: The roof leaks. = Le toit fuit. 

Стало: We're all afraid. ==> u sommes toutes mortes.                                   
Было: We're all afraid. = Nous avons toutes peur. 

Стало: I like your legs. ==> 'aime te sent.                                            
Было: I like your legs. = J'aime vos jambes. 

